In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import paper_vrp_mini as research

# for plotting
from paper_vrp_mini import visuals
visuals.set_visuals()


In [ ]:
# display plots inline
%matplotlib inline

In [ ]:
# ResearchData provides the database for stuff related to the paper;
# initialize it with a correct path
hangar = research.ResearchData("../vrp_paper_hangar_mini.h5")

# Example 1: covariance matrix from a row of variances

In [ ]:
# parameters
horizon_s = "3m"
counter_currency = "usd"
row_idx = "2015-01-14"

In [ ]:
# fetch MFIV from the database, take one row
mfiv = hangar.get("mfiv/m" + horizon_s)
mfiv_row = mfiv.loc[row_idx]

In [ ]:
# calculate
mficov = research.fx_covmat_from_variances(mfiv_row, 
                                           counter_currency)

In [ ]:
# plot as heatmap
fig, ax = plt.subplots(figsize=visuals.figsize_2, facecolor='w')
sns.heatmap(mficov.pow(1/2)*100, 
            ax=ax, annot=True, fmt=".2f", square=False, 
            cmap=visuals.heatmap_cmap)

# Example 2: from covariance matrix to variance of a currency portfolio

In [ ]:
# define weights of currencies
portfolio = pd.Series({"aud": 1/3, "nzd": 1/3, "cad": 1/3, 
                       "jpy": -1/3, "chf": -1/3, "eur": -1/3})

# reindex to follow the layout of the covariance matrix
portfolio = portfolio.reindex(mficov.columns, fill_value=0.0)

# convert to CovarianceDataFrame for easier manipulations
mficov_c = research.CovarianceDataFrame(mficov)

# calculate variance as a quadratic form
portfolio_var = mficov_c.quadratic_form(other=portfolio,
                                        trim=True)

In [ ]:
# plot the portfolio variance along individual variances 
# (i.e. variances of appreciation rates vs. USD)
to_plot = mficov_c.get_variances()\
    .assign(carry=portfolio_var)\
    .iloc[0]\
    .pow(1/2) * 100
    
fig, ax = plt.subplots(figsize=visuals.figsize_1, facecolor='w')
to_plot.plot(ax=ax, linestyle="none", marker='o')
ax.set_xticks(range(len(to_plot)))
ax.set_xticklabels(to_plot.index)